In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date
import datetime
import numpy as np
import seaborn as sns
import re
import json
import pandas as pd
import math
import ast
import os
from collections import defaultdict
import sys
sys.path.append('..')
from utilities import logger
from utilities import configuration
from utilities import health_data

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.preprocessing import StandardScaler

from sklearn.neural_network import MLPClassifier

from scipy import stats
from scipy import sparse
from sklearn.metrics import roc_auc_score, confusion_matrix

In [2]:
params = {'fix_missing_in_testing': True}

training ,testing = health_data.Admission.get_training_testing_data(filtering=True)
if params['fix_missing_in_testing']:
    for admission in testing:
        admission.fix_missings(training)

print(f'{len(training):,}')
print(f'{len(testing):,}')
print(f'{len(training)+len(testing):,}')

419,139
104,884
524,023


In [21]:

list_variable_description(training=training,testing=testing,logging=None,config=None)

,Variable,Avg Length,Min,Max,Unique,Unique (in training),Mode,Mode Count,Less Frequent,Less Frequent Count,\% Empty
0,Diagnosis Texts,4.452396,0,50,9907,9537,benign hypertension,79878,"fracture of acromial process of scapula, open",1,0.004389
1,Diagnosis Codes,4.452415,0,50,9921,9545,i100,79878,s42101,1,0.004389
2,Diagnosis Types,4.452280,0,50,12,11,3,741855,4,1,0.008969
3,Intervention Codes,1.095591,0,50,7973,7519,3ip10vx,22666,1ta74danw,1,50.434046
4,Intervention Texts,1.095580,0,50,7773,7350,"xray, heart with coronary arteries of left hea...",22664,"repair, tendons of the arm [around shoulder] u...",1,50.434046


In [13]:
stats.mode([elem for list_ in list_of_lists for elem in list_])

TypeError: Argument `a` is not recognized as numeric. Support for input that cannot be coerced to a numeric array was deprecated in SciPy 1.9.0 and removed in SciPy 1.11.0. Please consider `np.unique`.

In [14]:
stats.mode

<function scipy.stats._stats_py.mode(a, axis=0, nan_policy='propagate', keepdims=False)>

In [3]:
cat_df, main_pt_vocab = health_data.Admission.categorical_features(training)
cat_df

,male,female,transfusion given,is alc,is central zone,elective admission,urgent admission,level 1 comorbidity,level 2 comorbidity,level 3 comorbidity,...,Haematology,Endocrinology,Newborn,Gastro-Enterology,Neonatology,Infectious Disease,Paediatric Gynaecology,OBS Aborted,Gynaecology,Transplant Surgery
0,0,1,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419134,1,0,0,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
419135,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
419136,1,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
419137,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
training[0].inte

In [16]:
(training[0]).px_long_text

<Admission Patient_code='2232961' admit='2014-12-30' discharged='2015-01-01' Age='38' gender='Gender.FEMALE' ALC_days='0' acute_days='2' readmited(2015-01-03,2015-01-04,ReadmissionCode.UNPLANNED_READMIT_0_7)>

In [17]:
(training[0]).intervention_code

['1OD89DA']

In [18]:
(training[0]).px_long_text

['Excision total, gallbladder endoscopic [laparoscopic] approach without extraction (of calculi) chole']

In [22]:
min([len(admission.diagnosis.texts) for admission in training+testing])

0

In [ ]:
stats.mode([len(admission.diagnosis.texts) for admission in training+testing])

In [19]:
training[0].diagnosis.texts

['Calculus of gallbladder with other cholecystitis without mention of obstruction',
 'Tobacco use']

In [6]:
len(main_pt_vocab)

49

In [5]:
cat_df

,male,female,transfusion given,is alc,is central zone,elective admission,new born admission,urgent admission,level 1 comorbidity,level 2 comorbidity,...,OBS Aborted,Family Practice,Endocrinology,Gynaecology,Dentistry,Neurosurgery,Haematology,Neurology,Urology,Nephrology
0,0,1,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,1,0,0,1,1,0,...,0,1,0,0,0,0,0,0,0,0
3,0,1,0,0,1,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,1,0,0,1,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419134,1,0,0,0,0,0,0,1,1,1,...,0,1,0,0,0,0,0,0,0,0
419135,1,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
419136,1,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
419137,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
cat_df, main_pt_vocab = health_data.Admission.intervention_codes_features(training)


In [11]:
len(cat_df)

7519

In [3]:
419139+104884

524023

In [5]:
419139+104884

524023

In [3]:
419988+104998

524986

In [14]:
config = configuration.get_config()

f = open(config['train_val_json'])
train_val_data = json.load(f)

In [15]:
train_val_data['0'].keys()

dict_keys(['HCN code', 'Institution Number', 'Admit Date', 'Discharge Date', 'Readmission Code', 'Patient Age', 'Gender', 'MRDx', 'Postal Code', 'Diagnosis Code', 'Diagnosis Long Text', 'Diagnosis Type', 'Intervention Code', 'Px Long Text', 'Admit Category', 'Entry Code', 'Transfusion Given', 'Main Pt Service', 'CMG', 'Comorbidity Level', 'Case Weight', 'ALC Days', 'Acute Days', 'Institution To', 'Institution From', 'Institution Type', 'Discharge Nurse Unit', 'CZ Status'])

In [8]:
f = open(config['train_val_json'])
train_val_data = json.load(f)

# ---------- ---------- ---------- ---------- 
# Converting JSON to DataClasses
# ---------- ---------- ---------- ---------- 
all_admissions = []
for ix in train_val_data:
    all_admissions.append(
        health_data.Admission.from_dict_data(admit_id=int(ix), admission=train_val_data[ix])
        )

In [9]:
set([train_val_data[key]['Entry Code'] for key in train_val_data])

{'C Clinic from report',
 'D Direct',
 'E Emergency Departme',
 'N Newborn',
 None,
 'P Day Surgery from r',
 'S Stillborn'}

In [13]:
len([str(admission.entry_code) for admission in all_admissions if str(admission.entry_code)=='EntryCode.NONE'])

1

In [6]:
config = configuration.get_config()
for key in config:
    print(f'{key:40} (type: {type(config[key])})')


logging = logger.init_logger(config['system_log'])
logging.debug('Logger has started ont notebook 09 Random sample of instances.ipynb ...')

system_log                               (type: <class 'str'>)
logreg_log                               (type: <class 'str'>)
creating_database_log                    (type: <class 'str'>)
creating_held_out_log                    (type: <class 'str'>)
building_diag_interv_map_log             (type: <class 'str'>)
logreg_results                           (type: <class 'str'>)
logreg_latex                             (type: <class 'str'>)
latex_tables                             (type: <class 'str'>)
figures_folder                           (type: <class 'str'>)
json_file                                (type: <class 'str'>)
train_val_json                           (type: <class 'str'>)
heldout_json                             (type: <class 'str'>)
unused_after_heldout_json                (type: <class 'str'>)
unified_merged_file_cz                   (type: <class 'str'>)
unified_merged_file_noncz                (type: <class 'str'>)
unified_merged_file                      (type: <class 

In [11]:
training ,testing = health_data.Admission.get_training_testing_data(filtering=True)
print(len(training))
print(len(testing))
print(len(training)+len(testing))

419139
104884
524023


In [6]:
# all_admissions = [admission for admission in training+testing if admission.is_valid_testing_instance]
# len(all_admissions)
set([str(admission.entry_code) for admission in training+testing])

{'EntryCode.CLINIC_ENTRY',
 'EntryCode.DAY_SURGERY_ENTRY',
 'EntryCode.DIRECT_ENTRY',
 'EntryCode.EMERGENCY_ENTRY',
 'EntryCode.NEWBORN_ENTRY'}

In [7]:
all_admissions = [admission for admission in training+testing if admission.is_valid_testing_instance]

In [9]:
len([admission for admission in all_admissions if admission.entry_code == health_data.EntryCode.NONE])

524023

In [11]:
config = configuration.get_config()

# ---------- ---------- ---------- ---------- 
# Retriving train testing data from JSON file
# ---------- ---------- ---------- ---------- 
f = open(config['train_val_json'])
train_val_data = json.load(f)


In [14]:
len([train_val_data[key]['Entry Code'] for key in train_val_data if train_val_data[key]['Entry Code'] is None])

1

In [2]:
params = {'fix_missing_in_testing': True}

training ,testing = health_data.Admission.get_training_testing_data(filtering=False)
if params['fix_missing_in_testing']:
    for admission in testing:
        admission.fix_missings(training)


KeyboardInterrupt: 

In [ ]:
num_fields = [
          'age',
          'alc_days',
          'acute_days',
          'cmg',
          'case_weight',
]
field_names = ['Age',
               'ALC Days',
               'Acute Days',
               'Case Mix Group',
               'Case Weight',
               ]

data = {'Mean':[],
        'Std':[],
        'Min':[],
        'Q1':[],
        'Median':[],
        'Q3':[],
        'Max':[],
        'Kurtosis':[],
        'Skew':[],
        'Mode':[],
        'Mode Count':[],
        'Missing Count (%)': [],
        }
for field in num_fields:
    numbers = [getattr(admission, field) for admission in training+testing]
    missing_count = len([_ for num in numbers if num is None or np.isnan(num)])
    data['Missing Count (%)'].append(100*(missing_count/len(training+testing)))
    numbers = list(filter(lambda num: not num is None and not np.isnan(num), numbers))
    data['Mean'].append(np.average(numbers))
    data['Std'].append(np.std(numbers))
    data['Min'].append(np.min(numbers))
    data['Max'].append(np.max(numbers))
    data['Median'].append(np.median(numbers))
    data['Mode'].append(stats.mode(numbers)[0])
    data['Mode Count'].append(stats.mode(numbers)[1])
    data['Kurtosis'].append(stats.kurtosis(numbers))
    data['Skew'].append(stats.skew(numbers))
    data['Q1'].append(np.quantile(numbers,0.25))
    data['Q3'].append(np.quantile(numbers,0.75))
df = pd.DataFrame(data, index=field_names)
# print(df)
print(df.to_latex(float_format=f"{{:0.3f}}".format))
df

In [ ]:
fields = ['code',
          'admit_date',
          'discharge_date',
          'gender',
          'postal_code',
          'is_central_zone',
          'institution_number',
          'institution_to',
          'institution_from',
          'institution_type',
          'admit_category',
          'readmission_code',
          'main_pt_service',
          'mrdx',
          'transfusion_given',]

data = {
        'Levels':[],
        'Levels (in training)':[],
        'Mode':[],
        'Mode Count':[],
        'Less Frequent':[],
        'Less Frequent Count':[],
        'Missings (%)':[],
}

for field in fields:
    vector = [getattr(admission,field) for admission in training+testing]
    if field == 'gender':
        miss_count = len([_ for elem in vector if elem is None or elem==health_data.Gender.NONE])
        vector = list(filter(lambda elem: not elem is None and elem!=health_data.Gender.NONE  ,vector))
    elif field == 'admit_category':
        miss_count = len([_ for elem in vector if elem is None or elem==health_data.AdmitCategory.NONE])
        vector = list(filter(lambda elem: not elem is None and elem!=health_data.AdmitCategory.NONE  ,vector))
    elif field == 'readmission_code':
        miss_count = len([_ for elem in vector if elem is None or elem==health_data.ReadmissionCode.NONE])
        vector = list(filter(lambda elem: not elem is None and elem!=health_data.ReadmissionCode.NONE  ,vector))
    elif field == 'transfusion_given':
        miss_count = len([_ for elem in vector if elem is None or elem==health_data.TransfusionGiven.NONE])
        vector = list(filter(lambda elem: not elem is None and elem!=health_data.TransfusionGiven.NONE  ,vector))
    else:
        miss_count = len([_ for elem in vector if elem is None])
        vector = list(filter(lambda elem: not elem is None,vector))
    data['Missings (%)'].append(100*(miss_count/len(training+testing)))
    data['Levels'].append(len(set(vector)))
    data['Mode'].append(pd.Series(vector).mode()[0])
    data['Mode Count'].append(len([elem for elem in vector if elem==data['Mode'][-1]]))
    # Computing Less freq
    freq = defaultdict(int)
    for elem in vector:
        freq[elem]+=1
    data['Less Frequent'].append(sorted(freq.items(), key=lambda key_value: key_value[1])[0][0])
    data['Less Frequent Count'].append(sorted(freq.items(), key=lambda key_value: key_value[1])[0][1])
    # print(f'{field:20} min: {sorted(freq.items(), key=lambda key_value: key_value[1])[0]}')
    # print(f'{field:20} max: {sorted(freq.items(), key=lambda key_value: key_value[1])[-1]}')

    # Count levels in training
    vector = [getattr(admission,field) for admission in training]
    if field == 'gender':
        miss_count = len([_ for elem in vector if elem is None or elem==health_data.Gender.NONE])
        vector = list(filter(lambda elem: not elem is None and elem!=health_data.Gender.NONE  ,vector))
    elif field == 'admit_category':
        miss_count = len([_ for elem in vector if elem is None or elem==health_data.AdmitCategory.NONE])
        vector = list(filter(lambda elem: not elem is None and elem!=health_data.AdmitCategory.NONE  ,vector))
    elif field == 'readmission_code':
        miss_count = len([_ for elem in vector if elem is None or elem==health_data.ReadmissionCode.NONE])
        vector = list(filter(lambda elem: not elem is None and elem!=health_data.ReadmissionCode.NONE  ,vector))
    elif field == 'transfusion_given':
        miss_count = len([_ for elem in vector if elem is None or elem==health_data.TransfusionGiven.NONE])
        vector = list(filter(lambda elem: not elem is None and elem!=health_data.TransfusionGiven.NONE  ,vector))
    else:
        miss_count = len([_ for elem in vector if elem is None])
        vector = list(filter(lambda elem: not elem is None,vector))
    data['Levels (in training)'].append(len(set(vector)))

pd.DataFrame(data, index=fields)

In [ ]:
# training ,testing = health_data.Admission.get_training_testing_data(filtering=False)
# print(f'BOTH:                      {len(training+testing)}')
# print(f'Training before filtering: {len(training):,}')
# print(f'testing before filtering:  {len(testing):,}')
# print()
# training = list(filter(lambda admission: not admission.code is None and \
#                        admission.admit_category!=health_data.AdmitCategory.CADAVER and \
#                        admission.admit_category!=health_data.AdmitCategory.STILLBORN,
#                        training))

# testing = list(filter(lambda admission: not admission.code is None and \
#                        admission.admit_category!=health_data.AdmitCategory.CADAVER and \
#                        admission.admit_category!=health_data.AdmitCategory.STILLBORN,
#                        testing))

# print(f'BOTH:                      {len(training+testing)}')
# print(f'Training after filtering:  {len(training):,}')
# print(f'testing after filtering:   {len(testing):,}')
# print()

# training ,testing = health_data.Admission.get_training_testing_data(filtering=True)

# print(f'BOTH:                      {len(training+testing)}')
# print(f'Training after filtering:  {len(training):,}')
# print(f'testing after filtering:   {len(testing):,}')
# print()

# # categorical_df, main_pt_services_list = health_data.Admission.categorical_features(training)
# numerical_df = health_data.Admission.numerical_features(training, 
#                                                         fix_skew=False, 
#                                                         normalize=False)
# stds = np.std(numerical_df)
# mean = np.mean(numerical_df, axis=0)
# is_outlier=np.sum(numerical_df.values > (mean+4*stds).values, axis=1)>0
# print(f'outlier count = {np.sum(is_outlier)}')
# mask=~is_outlier

# features = []
# # features.append(sparse.csr_matrix(categorical_df.values))
# features.append((numerical_df.values))

# X_train = sparse.hstack([matrix[mask,:] for matrix in features])
# print(X_train.shape)

Training instances before filtering: 419988
Training instances after filtering:  419139
Testomg instances before filtering:  104998
Testomg instances after filtering:   104884


### Categorical Variables

\begin{tabular}{lrrrrrrrrrrrr}
\toprule
 & Mean & Std & Min & Q1 & Median & Q3 & Max & Kurtosis & Skew & Mode & Mode Count & Missing Count (%) \\
\midrule
Age & 58.720 & 23.540 & -1.000 & 46.000 & 64.000 & 76.000 & 110.000 & 0.126 & -0.867 & 0.000 & 26192 & 0.000 \\
ALC Days & 1.831 & 29.951 & 0.000 & 0.000 & 0.000 & 0.000 & 16300.000 & 169867.239 & 327.838 & 0.000 & 506177 & 0.000 \\
Acute Days & 8.364 & 37.494 & 0.000 & 2.000 & 4.000 & 8.000 & 17343.000 & 109751.599 & 273.665 & 1.000 & 102153 & 0.000 \\
Case Mix Group & 411.392 & 245.462 & 1.000 & 196.000 & 337.000 & 635.000 & 999.000 & -1.263 & 0.270 & 139.000 & 16206 & 0.000 \\
Case Weight & 1.792 & 5.968 & 0.000 & 0.537 & 0.923 & 1.618 & 2000.000 & 48866.216 & 164.373 & 1.118 & 2183 & 0.000 \\
\bottomrule
\end{tabular}



,Mean,Std,Min,Q1,Median,Q3,Max,Kurtosis,Skew,Mode,Mode Count,Missing Count (%)
Age,58.719770,23.539856,-1.0,46.0000,64.0000,76.0000,110.0,0.126155,-0.866810,0.0000,26192,0.0
ALC Days,1.831414,29.951247,0.0,0.0000,0.0000,0.0000,16300.0,169867.239251,327.838350,0.0000,506177,0.0
Acute Days,8.363591,37.493524,0.0,2.0000,4.0000,8.0000,17343.0,109751.598524,273.664683,1.0000,102153,0.0
Case Mix Group,411.391888,245.461806,1.0,196.0000,337.0000,635.0000,999.0,-1.263073,0.270383,139.0000,16206,0.0
Case Weight,1.791766,5.967645,0.0,0.5373,0.9234,1.6184,2000.0,48866.216245,164.372782,1.1182,2183,0.0


### Nmuerical Variables

,Levels,Levels (in training),Mode,Mode Count,Less Frequent,Less Frequent Count,Missings (%)
code,263631,230674,170806,1022,4584715,1,0.000000
admit_date,2743,2717,2019-09-24 00:00:00,314,2014-03-26 00:00:00,1,0.000000
discharge_date,2466,2466,2015-04-02 00:00:00,362,2017-12-25 00:00:00,59,0.000000
gender,3,3,Gender.FEMALE,269931,Gender.OTHER,9,0.000000
postal_code,32613,30652,B0P1R0,5160,B2N7H9,1,0.000000
is_central_zone,2,2,False,306328,True,217695,0.000000
institution_number,34,33,85,172834,58,10,0.000000
institution_to,249,244,HOME CARE SERVICES-HOME CARE,19320,South Cumberland Hospital,1,70.622473
institution_from,289,286,29087,15832,ST. MARTHA'S REGIONAL HOSPITAL-INPT REHAB,1,68.530198
institution_type,28,28,"OTHER (9002,9003,9088,9089)",20583,NACRS AMBULATORY CARE (9),1,85.160575


In [21]:
# freq = defaultdict(int)
# vec = [1,2,2,3,3,4,4]
# list(map(lambda elem: freq[elem]+1, vec))
# freq

defaultdict(int, {1: 0, 2: 0, 3: 0, 4: 0})

#### MISSING VALUES

In [27]:
# # training ,testing = health_data.Admission.get_training_testing_data(filtering=False)

# field = 'code'
# missing_count = len([_ for admission in training+testing if getattr(admission, field) is None])
# print(f'{field:20} = {missing_count/len(training+testing):.3%}')

# field = 'admit_date'
# missing_count = len([_ for admission in training+testing if getattr(admission, field) is None])
# print(f'{field:20} = {missing_count/len(training+testing):.3%}')

# field = 'discharge_date'
# missing_count = len([_ for admission in training+testing if getattr(admission, field) is None])
# print(f'{field:20} = {missing_count/len(training+testing):.3%}')

# field = 'age'
# missing_count = len([_ for admission in training+testing if getattr(admission, field) is None or np.isnan(getattr(admission,field))])
# print(f'{field:20} = {missing_count/len(training+testing):.3%}')

# field = 'alc_days'
# missing_count = len([_ for admission in training+testing if getattr(admission, field) is None or np.isnan(getattr(admission,field))])
# print(f'{field:20} = {missing_count/len(training+testing):.3%}')

# field = 'acute_days'
# missing_count = len([_ for admission in training+testing if getattr(admission, field) is None or np.isnan(getattr(admission,field))])
# print(f'{field:20} = {missing_count/len(training+testing):.3%}')

# field = 'cmg'
# missing_count = len([_ for admission in training+testing if getattr(admission, field) is None or np.isnan(getattr(admission,field))])
# print(f'{field:20} = {missing_count/len(training+testing):.3%}')

# field = 'case_weight'
# missing_count = len([_ for admission in training+testing if getattr(admission, field) is None or np.isnan(getattr(admission,field))])
# print(f'{field:20} = {missing_count/len(training+testing):.3%}')

# field = 'gender'
# missing_count = len([_ for admission in training+testing if getattr(admission, field) is None or \
#                                                                 getattr(admission,field)==health_data.Gender.NONE])
# print(f'{field:20} = {missing_count/len(training+testing):.3%}')

# field = 'postal_code'
# missing_count = len([_ for admission in training+testing if getattr(admission, field) is None or \
#                                                                 getattr(admission,field)=='x9x9x9'])
# print(f'{field:20} = {missing_count/len(training+testing):.3%}')

# field = 'is_central_zone'
# missing_count = len([_ for admission in training+testing if getattr(admission, field) is None ])
# print(f'{field:20} = {missing_count/len(training+testing):.3%}')

# field = 'institution_number'
# missing_count = len([_ for admission in training+testing if getattr(admission, field) is None or np.isnan(getattr(admission, field)) ])
# print(f'{field:20} = {missing_count/len(training+testing):.3%}')

# field = 'institution_to'
# missing_count = len([_ for admission in training+testing if getattr(admission, field) is None ])
# print(f'{field:20} = {missing_count/len(training+testing):.3%}')

# field = 'institution_from'
# missing_count = len([_ for admission in training+testing if getattr(admission, field) is None ])
# print(f'{field:20} = {missing_count/len(training+testing):.3%}')

# field = 'institution_type'
# missing_count = len([_ for admission in training+testing if getattr(admission, field) is None])
# print(f'{field:20} = {missing_count/len(training+testing):.3%}')

# field = 'admit_category'
# missing_count = len([_ for admission in training+testing if getattr(admission, field) is None or \
#                                                                 getattr(admission,field)==health_data.AdmitCategory.NONE])
# print(f'{field:20} = {missing_count/len(training+testing):.3%}')

# field = 'readmission_code'
# missing_count = len([_ for admission in training+testing if getattr(admission, field) is None or \
#                                                                 getattr(admission,field)==health_data.ReadmissionCode.NONE])
# print(f'{field:20} = {missing_count/len(training+testing):.3%}')

# field = 'main_pt_service'
# missing_count = len([_ for admission in training+testing if getattr(admission, field) is None or getattr(admission, field) ==[]])
# print(f'{field:20} = {missing_count/len(training+testing):.3%}')

# field = 'mrdx'
# missing_count = len([_ for admission in training+testing if getattr(admission, field) is None])
# print(f'{field:20} = {missing_count/len(training+testing):.3%}')

# field = 'transfusion_given'
# missing_count = len([_ for admission in training+testing if getattr(admission, field) is None or \
#                                                                 getattr(admission,field)==health_data.TransfusionGiven.NONE])
# print(f'{field:20} = {missing_count/len(training+testing):.3%}')

# field = 'intervention_code'
# missing_count = len([_ for admission in training+testing if getattr(admission, field) is None])
# print(f'{field:20} = {missing_count/len(training+testing):.3%}')

# field = 'diagnosis_types'
# missing_count = len([_ for admission in training+testing if getattr(admission, 'diagnosis').types is None])
# print(f'{field:20} = {missing_count/len(training+testing):.3%}')

# field = 'diagnosis_texts'
# missing_count = len([_ for admission in training+testing if getattr(admission, 'diagnosis').texts is None])
# print(f'{field:20} = {missing_count/len(training+testing):.3%}')

# field = 'diagnosis_codes'
# missing_count = len([_ for admission in training+testing if getattr(admission, 'diagnosis').codes is None])
# print(f'{field:20} = {missing_count/len(training+testing):.3%}')

code                 = 0.111%
admit_date           = 0.001%
discharge_date       = 0.000%
age                  = 0.000%
alc_days             = 0.000%
acute_days           = 0.000%
cmg                  = 0.020%
case_weight          = 0.020%
gender               = 0.001%
postal_code          = 0.000%
is_central_zone      = 0.000%
institution_number   = 0.000%
institution_to       = 70.643%
institution_from     = 68.536%
institution_type     = 85.172%
admit_category       = 0.000%
readmission_code     = 0.087%
main_pt_service      = 0.018%
mrdx                 = 0.000%
transfusion_given    = 0.019%
intervention_code    = 0.000%
diagnosis_types      = 0.000%
diagnosis_texts      = 0.000%
diagnosis_codes      = 0.000%


#### UNIQUE VALUE COUNT

In [40]:

# fields = ['code',
#           'admit_date',
#           'discharge_date',]


# for field in fields:
#     count_unique = len(set([getattr(admission, field) for admission in training+testing]))
#     print(f'{field:20} = {count_unique:>,}')

# fields = [
#           'age',
#           'alc_days',
#           'acute_days',
#           'cmg',
#           'case_weight',
# ]
# for field in fields:
#     min_ = (min([getattr(admission, field) for admission in training+testing if not getattr(admission, field) is None and not np.isnan(getattr(admission, field))]))
#     max_ = (max([getattr(admission, field) for admission in training+testing if not getattr(admission, field) is None and not np.isnan(getattr(admission, field))]))
#     print(f'{field:20} = \t\t\t\t{min_:>,}\t\t\t\t{max_}')

# fields = [
#           'gender',
#           'postal_code',
#           'is_central_zone',
#           'institution_number',
#           'institution_to',
#           'institution_from',
#           'institution_type',
#           'admit_category',
#           'readmission_code',
#           'main_pt_service',
#           'mrdx',
#           'transfusion_given',
#           ]

# for field in fields:
#     count_unique = len(set([getattr(admission, field) for admission in training+testing]))
#     print(f'{field:20} = {count_unique:>,}')

# fields = [
#           'intervention_code',
#           'diagnosis_types',
#           'diagnosis_texts',
#           'diagnosis_codes',
# ]


code                 = 263,719
admit_date           = 2,747
discharge_date       = 2,466
age                  = 				-1				110
alc_days             = 				0				16300
acute_days           = 				0				17343
cmg                  = 				1.0				999.0
case_weight          = 				0.0				2000.0
gender               = 5
postal_code          = 32,769
is_central_zone      = 2
institution_number   = 34
institution_to       = 250
institution_from     = 290
institution_type     = 29
admit_category       = 6
readmission_code     = 7
main_pt_service      = 53
mrdx                 = 6,462
transfusion_given    = 3


In [ ]:
for admission in training+testing:
    

### CROSS TAB

In [22]:
categorical_df, main_pt_services_list = health_data.Admission.categorical_features(training)
y = health_data.Admission.get_y(training)
categorical_df['Target']=y
categorical_df.columns[:13]

,male,female,transfusion given,is alc,is central zone,elective admission,new born admission,urgent admission,level 1 comorbidity,level 2 comorbidity,...,OBS Aborted,Dermatology,Neonatology,Neurology,Gastro-Enterology,Geriatrics,Cardiology,Paediatric Traumatology,Gynaecology,Target
0,0,1,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419134,1,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
419135,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
419136,1,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
419137,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [78]:
fields = categorical_df.columns[:4]


df = pd.crosstab(categorical_df['Target'],
            [categorical_df[field_name] for field_name in fields], 
            colnames=[field_name for field_name in fields], 
            # margins=True,
            # normalize=True,
            )

df.iloc[0,:] = 100*(df.iloc[0,:]/np.sum(df.iloc[0,:]))
df.iloc[1,:] = 100*(df.iloc[1,:]/np.sum(df.iloc[1,:]))

df['All']=['100%','100%',]
df

'<table border="1" class="dataframe">\n  <thead>\n    <tr>\n      <th>male</th>\n      <th colspan="5" halign="left">0</th>\n      <th colspan="4" halign="left">1</th>\n      <th>All</th>\n    </tr>\n    <tr>\n      <th>female</th>\n      <th>0</th>\n      <th colspan="4" halign="left">1</th>\n      <th colspan="4" halign="left">0</th>\n      <th></th>\n    </tr>\n    <tr>\n      <th>transfusion given</th>\n      <th>0</th>\n      <th colspan="2" halign="left">0</th>\n      <th colspan="2" halign="left">1</th>\n      <th colspan="2" halign="left">0</th>\n      <th colspan="2" halign="left">1</th>\n      <th></th>\n    </tr>\n    <tr>\n      <th>is alc</th>\n      <th>0</th>\n      <th>0</th>\n      <th>1</th>\n      <th>0</th>\n      <th>1</th>\n      <th>0</th>\n      <th>1</th>\n      <th>0</th>\n      <th>1</th>\n      <th></th>\n    </tr>\n    <tr>\n      <th>Target</th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th

In [79]:
fields = categorical_df.columns[4:8]


df = pd.crosstab(categorical_df['Target'],
            [categorical_df[field_name] for field_name in fields], 
            colnames=[field_name for field_name in fields], 
            # margins=True,
            # normalize=True,
            )

df.iloc[0,:] = 100*(df.iloc[0,:]/np.sum(df.iloc[0,:]))
df.iloc[1,:] = 100*(df.iloc[1,:]/np.sum(df.iloc[1,:]))

df['All']=['100%','100%',]
df

is central zone             0                               1            \
elective admission          0                    1          0             
new born admission          0         1          0          0         1   
urgent admission            1         0          0          1         0   
Target                                                                    
0                   38.567521  4.684906  15.385190  30.089586  0.002987   
1                   48.138145  0.965406   6.028043  37.811746  0.005746   

is central zone                 All  
elective admission          1        
new born admission          0        
urgent admission            0        
Target                               
0                   11.269811  100%  
1                    7.050914  100%

In [80]:
fields = categorical_df.columns[8:12]


df = pd.crosstab(categorical_df['Target'],
            [categorical_df[field_name] for field_name in fields], 
            colnames=[field_name for field_name in fields], 
            # margins=True,
            # normalize=True,
            )

df.iloc[0,:] = 100*(df.iloc[0,:]/np.sum(df.iloc[0,:]))
df.iloc[1,:] = 100*(df.iloc[1,:]/np.sum(df.iloc[1,:]))

df['All']=['100%','100%',]
df

level 1 comorbidity          0          1                                  All
level 2 comorbidity          0          0          1                          
level 3 comorbidity          0          0          0         1                
level 4 comorbidity          0          0          0         0         1      
Target                                                                        
0                    68.444032  10.968868   9.554510  6.428833  4.603758  100%
1                    57.786461  15.503965  13.372026  8.487530  4.850017  100%

In [101]:
params = {'fix_skew': False,
            'normalize': False,
            'fix_missing_in_testing': True,
            'numerical_features': True,
            'categorical_features': True,
            'diagnosis_features': True,
            'intervention_features':True,
            'use_idf':False,
            'class_balanced':True,
            'remove_outliers': False,
        }

In [97]:
features = []

if params['numerical_features']:
    numerical_df = health_data.Admission.numerical_features(training,)
    
    if params['remove_outliers']:
        stds = np.std(numerical_df)
        mean = np.mean(numerical_df, axis=0)
        is_outlier=np.sum(numerical_df.values > (mean+4*stds).values, axis=1)>0
    
    if params['fix_skew']:
        numerical_df['case_weight'] = np.log10(numerical_df['case_weight']+1)
        numerical_df['acute_days'] = np.log10(numerical_df['acute_days']+1)
        numerical_df['alc_days'] = np.log10(numerical_df['alc_days']+1)

    if params['normalize']:
        scaler = StandardScaler()
        if params['remove_outliers']:
            scaler.fit(numerical_df.values[~is_outlier,:])
        else:
            scaler.fit(numerical_df.values)
        numerical_df = pd.DataFrame(scaler.transform(numerical_df.values), columns=numerical_df.columns)

    features.append(sparse.csr_matrix(numerical_df.values))

if params['categorical_features']:
    categorical_df, main_pt_services_list = health_data.Admission.categorical_features(training)
    features.append(sparse.csr_matrix(categorical_df.values))

if params['diagnosis_features']:
    vocab_diagnosis, diagnosis_matrix = health_data.Admission.diagnosis_codes_features(training, 
                                                                                    use_idf=params['use_idf'])
    features.append(diagnosis_matrix)

if params['intervention_features']:
    vocab_interventions, intervention_matrix = health_data.Admission.intervention_codes_features(training, 
                                                                                                use_idf=params['use_idf'])
    features.append(intervention_matrix)

if params['remove_outliers']:
    mask=~is_outlier
else:
    mask = np.ones(shape=(len(training)))==1

X_train = sparse.hstack([matrix[mask,:] for matrix in features])
y_train = health_data.Admission.get_y(training)[mask]
print(X_train.shape)
print(y_train.shape)

(419139, 17129)
(419139,)


In [89]:
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel

logreg = LogisticRegression(max_iter=7000, class_weight='balanced').fit(X_train, y_train)
model = SelectFromModel(logreg, prefit=True)
X_new = model.transform(X_train)
X_new.shape

(419139, 4875)

In [98]:
# ---------- ---------- ---------- ---------- ---------- ---------- ---------- ---------- 
# TESTING MATRIX
# ---------- ---------- ---------- ---------- ---------- ---------- ---------- ---------- 
features = []
if params['numerical_features']:
    numerical_df = health_data.Admission.numerical_features(testing,)
    
    if params['fix_skew']:
        numerical_df['case_weight'] = np.log10(numerical_df['case_weight']+1)
        numerical_df['acute_days'] = np.log10(numerical_df['acute_days']+1)
        numerical_df['alc_days'] = np.log10(numerical_df['alc_days']+1)

    if params['normalize']:
        numerical_df = pd.DataFrame(scaler.transform(numerical_df.values), columns=numerical_df.columns)
    features.append(sparse.csr_matrix(numerical_df.values))

if params['categorical_features']:
    categorical_df,_ = health_data.Admission.categorical_features(testing, main_pt_services_list=main_pt_services_list)
    features.append(sparse.csr_matrix(categorical_df.values))

if params['diagnosis_features']:
    vocab_diagnosis, diagnosis_matrix = health_data.Admission.diagnosis_codes_features(testing, 
                                                                                    vocabulary=vocab_diagnosis, 
                                                                                    use_idf=params['use_idf'])
    features.append(diagnosis_matrix)

if params['intervention_features']:
    vocab_interventions, intervention_matrix = health_data.Admission.intervention_codes_features(testing, 
                                                                                                vocabulary=vocab_interventions, 
                                                                                                use_idf=params['use_idf']
                                                                                                )
    features.append(intervention_matrix)

X_test = sparse.hstack(features)
y_test = health_data.Admission.get_y(testing)


In [99]:
# X_train = model.transform(X_train)
# X_test = model.transform(X_test)
select_best=SelectKBest(chi2, k=20)
X_train = select_best.fit_transform(X_train, y_train)
X_test = select_best.transform(X_test,)

print(X_train.shape)
print(X_test.shape)

(419139, 20)
(104884, 20)


In [100]:


logging.debug(f'X_train.shape = ({X_train.shape[0]:,} x {X_train.shape[1]:,})')
logging.debug(f'y_train.shape = ({y_train.shape[0]:,} x )')
# print()
logging.debug(f'X_test.shape =  ({X_test.shape[0]:,} x {X_test.shape[1]:,})')
logging.debug(f'y_test.shape =  ({y_test.shape[0]:,} x )')


# ---------- ---------- ---------- ---------- ---------- ---------- ---------- ----------
# LOGISTIC REGRESSION MODEL 
# ---------- ---------- ---------- ---------- ---------- ---------- ---------- ---------- 
class_weight = 'balanced' if params['class_balanced'] else None
clf = LogisticRegression(class_weight=class_weight, max_iter=7000,).fit(X_train, y_train,)

# ---------- ---------- ---------- ---------- ---------- ---------- ---------- ----------
# RESULTS (METRICS)
# ---------- ---------- ---------- ---------- ---------- ---------- ---------- ---------- 
y_true = y_train
y_pred = clf.predict(X_train)
y_score= clf.predict_proba(X_train)

model_name = str(clf)
columns = ['Model','split','TN','FP','FN','TP','Precision','Recall','F1-Score','AUC']
param_names  = [key for key in params.keys()]
columns = columns[0:1] + param_names + columns[1:]
str_ = ';'.join(columns)
logging.debug(str_)

tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
str_ = f'{model_name};{str(params)};TRAIN;{tn};{fp};{fn};{tp};{precision_score(y_true, y_pred,)};{recall_score(y_true, y_pred,)};'\
    f'{f1_score(y_true, y_pred,)};{roc_auc_score(y_true=y_true, y_score=y_pred)}'
logging.debug(str_)


vec1 = [model_name,
        'TRAIN',
        tn,
        fp,
        fn,
        tp,
        precision_score(y_true, y_pred,),
        recall_score(y_true, y_pred,),
        f1_score(y_true, y_pred,),
        roc_auc_score(y_true=y_true, y_score=y_pred),
        ]
vec1 = vec1[0:1] + [params[param_name] for param_name in param_names] + vec1[1:]

y_true = y_test
y_pred = clf.predict(X_test)
y_score= clf.predict_proba(X_test)

tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
str_ = f'{model_name};{str(params)};TEST;{tn};{fp};{fn};{tp};{precision_score(y_true, y_pred,)};{recall_score(y_true, y_pred,)};'\
    f'{f1_score(y_true, y_pred,):};{roc_auc_score(y_true=y_true, y_score=y_pred)}'
logging.debug(str_)                         

vec2 = [model_name,
        'TEST',
        tn,
        fp,
        fn,
        tp,
        precision_score(y_true, y_pred,),
        recall_score(y_true, y_pred,),
        f1_score(y_true, y_pred,),
        roc_auc_score(y_true=y_true, y_score=y_pred),
        ]
vec2 = vec2[0:1] + [params[param_name] for param_name in param_names] + vec2[1:]
m = np.vstack([vec1, vec2])
df = pd.DataFrame(m, columns=columns)
df

2023-11-29 10:36:59,936 - root - DEBUG - X_train.shape = (419,139 x 20)
2023-11-29 10:36:59,938 - root - DEBUG - y_train.shape = (419,139 x )
2023-11-29 10:36:59,938 - root - DEBUG - X_test.shape =  (104,884 x 20)
2023-11-29 10:36:59,939 - root - DEBUG - y_test.shape =  (104,884 x )
2023-11-29 10:37:11,737 - root - DEBUG - Model;fix_skew;normalize;fix_missing_in_testing;numerical_features;categorical_features;diagnosis_features;intervention_features;use_idf;class_balanced;remove_outliers;split;TN;FP;FN;TP;Precision;Recall;F1-Score;AUC
2023-11-29 10:37:12,256 - root - DEBUG - LogisticRegression(class_weight='balanced', max_iter=7000);{'fix_skew': False, 'normalize': False, 'fix_missing_in_testing': True, 'numerical_features': True, 'categorical_features': True, 'diagnosis_features': True, 'intervention_features': True, 'use_idf': False, 'class_balanced': True, 'remove_outliers': False};TRAIN;150041;251696;2781;14621;0.0549007385934807;0.8401907826686588;0.10306676676570832;0.60683572145

,Model,fix_skew,normalize,fix_missing_in_testing,numerical_features,categorical_features,diagnosis_features,intervention_features,use_idf,class_balanced,remove_outliers,split,TN,FP,FN,TP,Precision,Recall,F1-Score,AUC
0,"LogisticRegression(class_weight='balanced', ma...",False,False,True,True,True,True,True,False,True,False,TRAIN,150041,251696,2781,14621,0.0549007385934807,0.8401907826686588,0.10306676676570832,0.6068357214507987
1,"LogisticRegression(class_weight='balanced', ma...",False,False,True,True,True,True,True,False,True,False,TEST,37591,62760,750,3783,0.05685045759884586,0.8345466578424884,0.10644943440823906,0.604570914396227


In [93]:
from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2

X_new = SelectKBest(chi2, k=20).fit_transform(X_train, y_train)
X_new

<419139x20 sparse matrix of type '<class 'numpy.float64'>'
	with 394489 stored elements in Compressed Sparse Row format>

In [102]:
training ,testing = health_data.Admission.get_training_testing_data(filtering=True)
if params['fix_missing_in_testing']:
    for admission in testing:
        admission.fix_missings(training)


Training instances before filtering: 419988
Training instances after filtering:  419139
Testomg instances before filtering:  104998
Testomg instances after filtering:   104884


In [104]:
print(min([admission.discharge_date for admission in training]))
print(max([admission.discharge_date for admission in training]))

2015-01-01 00:00:00
2021-10-01 00:00:00


In [105]:
print(min([admission.discharge_date for admission in testing]))
print(max([admission.discharge_date for admission in testing]))

2015-01-01 00:00:00
2021-10-01 00:00:00


In [107]:
vocab, matrix = health_data.Admission.intervention_codes_features(training)

matrix

<419139x7519 sparse matrix of type '<class 'numpy.float64'>'
	with 444678 stored elements in Compressed Sparse Row format>

In [5]:

vocab[-3:]

array(['z994', 'z998', 'z999'], dtype=object)

In [6]:
diag_vocab, dianogis_df = health_data.Admission.diagnosis_codes_features(training)
diagcode2description = health_data.Admission.get_diagnoses_mapping()

for code in diag_vocab[:2]:
    if code in diagcode2description:
        print(diagcode2description[code])
    else:
        code = code.upper()
        print(f'{code:6}: {diagcode2description[code]}')
print('...')
for code in diag_vocab[-2:]:
    if code in diagcode2description:
        print(diagcode2description[code])
    else:
        code = code.upper()
        print(f'{code:6}: {diagcode2description[code]}')

A000  : {'Cholera due to Vibrio cholerae 01, biovar cholerae'}
A010  : {'Typhoid fever'}
...
Z998  : {'Dependence on other enabling machines and devices'}
Z999  : {'Dependence on unspecified enabling machine and device'}


In [10]:
interv_vocab, intervention_df = health_data.Admission.intervention_codes_features(training)
intervcode2description = health_data.Admission.get_intervention_mapping()

for code in interv_vocab[:2]:
    if code in intervcode2description:
        print(intervcode2description[code])
    else:
        code = code.upper()
        print(f'{code:6}: {intervcode2description[code]}')
print('...')
for code in interv_vocab[-2:]:
    if code in intervcode2description:
        print(intervcode2description[code])
    else:
        code = code.upper()
        print(f'{code:6}: {intervcode2description[code]}')

1AA13HAC2: {'Control of bleeding, meninges and dura mater of brain using percutaneous (needle) injection of antih'}
1AA13SEC2: {'Control of bleeding, meninges and dura mater of brain using injection of antihemorrhagic agent into '}
...
8ZZ70GRBW: {'Immunization (to prevent) immune disorder NEC by intravenous [IV] injection of immune globulin [IGIV'}
8ZZ70HABW: {'Immunization (to prevent) immune disorder NEC by intramuscular [IM] injection of (passive) immunoglo'}


In [17]:
# f = open(config['json_file'], encoding='utf-8')
# all_data = json.load(f)

for key, value in all_data['0'].items():
    print(f'{key:30}: {str(value)[:80]}')

HCN code                      : 1159480
Institution Number            : 65
Admit Date                    : 2014-12-24 00:00:00
Discharge Date                : 2015-01-01 00:00:00
Readmission Code              : 5 New patient to the acute care unit
Patient Age                   : 67
Gender                        : Male
MRDx                          : K746  Other and unspecified cirrhosis of liver
Postal Code                   : B3A2K1
Diagnosis Code                : ['K746', 'K431', 'R18', 'K760', 'K650', 'I518', 'L988', 'L984', 'K439', 'J90']
Diagnosis Long Text           : ['Other and unspecified cirrhosis of liver', 'Incisional hernia with gangrene', 
Diagnosis Type                : ['M', '1', 'W', '1', '1', '1', '1', '1', '1', '3']
Intervention Code             : ['1OT52HHD1']
Px Long Text                  : ['Drainage, abdominal cavity using percutaneous transcatheter approach and anti 
Admit Category                : U Emergent, urgent
Entry Code                    : E Emergency D

In [18]:

for key, value in all_data['0'].items():
    print(f'{key};{str(value)[:80]}')

HCN code;1159480
Institution Number;65
Admit Date;2014-12-24 00:00:00
Discharge Date;2015-01-01 00:00:00
Readmission Code;5 New patient to the acute care unit
Patient Age;67
Gender;Male
MRDx;K746  Other and unspecified cirrhosis of liver
Postal Code;B3A2K1
Diagnosis Code;['K746', 'K431', 'R18', 'K760', 'K650', 'I518', 'L988', 'L984', 'K439', 'J90']
Diagnosis Long Text;['Other and unspecified cirrhosis of liver', 'Incisional hernia with gangrene', 
Diagnosis Type;['M', '1', 'W', '1', '1', '1', '1', '1', '1', '3']
Intervention Code;['1OT52HHD1']
Px Long Text;['Drainage, abdominal cavity using percutaneous transcatheter approach and anti 
Admit Category;U Emergent, urgent
Entry Code;E Emergency Departme
Transfusion Given;Yes
Main Pt Service;Family Practice
CMG;285.0
Comorbidity Level;1  Level 1 Comorbidity
Case Weight;1.3894
ALC Days;0
Acute Days;8
Institution To;QE11 Health Sciences Centre
Institution From;Home Care NS - CIHI Disch Dispn 04, Dialysis, VON
Institution Type;ACUTE, ACUTE PS

In [16]:
type(all_data['0'])

dict

In [3]:
item = training[0]

In [4]:
item.entry_code

'E Emergency Departme'

In [6]:
set([admission.entry_code for admission in training+testing])

{'C Clinic from report',
 'D Direct',
 'E Emergency Departme',
 'N Newborn',
 'P Day Surgery from r'}

In [16]:
len(set([admission.main_pt_service for admission in training+testing]))

51

In [17]:
set([admission.main_pt_service for admission in training+testing])

{'<NONE>',
 'Allergy',
 'Alternate Level of Care',
 'Cardiology',
 'Cardiovascular Surgery',
 'Dentistry',
 'Dermatology',
 'Endocrinology',
 'Family Practice',
 'Gastro-Enterology',
 'General Medicine',
 'General Surgery',
 'Genetics',
 'Geriatrics',
 'Gynaecology',
 'Haematology',
 'Infectious Disease',
 'Neonatology',
 'Nephrology',
 'Neurology',
 'Neurosurgery',
 'Newborn',
 'OBS Aborted',
 'OBS Antepartum',
 'OBS Delivered',
 'OBS Postpartum',
 'Oncology',
 'Ophthalmology',
 'Oral Surgery',
 'Orthopaedic Surgery',
 'Otolaryngology and ORL',
 'Paediatric Dermatology',
 'Paediatric Endocrinology',
 'Paediatric Gynaecology',
 'Paediatric Medicine',
 'Paediatric Orthopaedic Surgery',
 'Paediatric Otolaryngology and ',
 'Paediatric Psychiatry',
 'Paediatric Surgery',
 'Paediatric Traumatology',
 'Palliative Care',
 'Plastic Surgery',
 'Psychiatry',
 'Radiotherapy',
 'Rehab in Acute Care Hospital',
 'Respirology',
 'Rheumatology',
 'Thoracic Surgery',
 'Transplant Surgery',
 'Traumatolo

In [3]:
(set([admission.entry_code for admission in training+testing]))

NameError: name 'training' is not defined

In [3]:
categorical_df, main_pt_services_list = health_data.Admission.categorical_features(training)
len(main_pt_services_list)

49

In [17]:
len(training)

419139

In [15]:
print(set([str(admission.entry_code) for admission in training]))
valid=0
for admission in training:
    if admission.is_valid_testing_instance:
        valid+=1
print(valid)
# set([str(admission.entry_code) for admission in training if admission.is_valid_testing_instance])
# set([str(admission.entry_code) for admission in training if admission.is_valid_testing_instance])

{'EntryCode.DAY_SURGERY_ENTRY', 'EntryCode.CLINIC_ENTRY', 'EntryCode.NEWBORN_ENTRY', 'EntryCode.EMERGENCY_ENTRY', 'EntryCode.DIRECT_ENTRY'}
419139


In [16]:
print(set([str(admission.entry_code) for admission in training]))

{'EntryCode.DAY_SURGERY_ENTRY', 'EntryCode.CLINIC_ENTRY', 'EntryCode.NEWBORN_ENTRY', 'EntryCode.EMERGENCY_ENTRY', 'EntryCode.DIRECT_ENTRY'}


In [2]:
params = {'fix_missing_in_testing': True}

training ,testing = health_data.Admission.get_training_testing_data(filtering=True)
# if params['fix_missing_in_testing']:
#     for admission in testing:
#         admission.fix_missings(training)
print(len(training))
print(len(testing))

Number of elements found: 524986
Number of entry type none: 7
Training before filtering: 419,988
Training after filtering:  0
Testing before filtering:  104,998
Testing after filtering:   104,884
0
104884


In [1]:
print(testing)

NameError: name 'testing' is not defined